<a href="https://colab.research.google.com/github/SkyRanger2010/DE2025_MS_HW2/blob/main/HomeWork2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import json # Added import for json

cookies = {
}

headers = {
    "Content-Type": "application/json" # Added content type
}

res = pd.DataFrame()
for rn in range(1, 26):
    # Fix: Construct a proper JSON payload for the page number
    # Assuming the API expects a JSON body with a 'page_number' key
    data = json.dumps({"page_number": rn}).encode('utf-8') # Corrected line

    response = requests.post('https://api.cian.ru/ebc-analytics/event-enrichment/', cookies=cookies, headers=headers,
                             data=data)

    products = response.json().get('products')
    if products is not None and len(products) > 0:
        res = pd.concat([res, pd.DataFrame().from_records(products)])
    else:
        print(f"No products found for page {rn}, skipping.")

def parse_property_info(text):
    result = {}

    # Общая площадь
    total_match = re.search(r'Общая площадь\s*([\d,]+)\s*м\u00b2', text)
    if total_match:
        result['total_area'] = total_match.group(1)

    # Площадь кухни
    kitchen_match = re.search(r'Площадь кухни\s*([\d,]+)\s*м\u00b2', text)
    if kitchen_match:
        result['kitchen_area'] = kitchen_match.group(1)

    # Этаж
    floor_match = re.search(r'Этаж\s*(\d+)\s*из\s*(\d+)', text)
    if floor_match:
        result['floor'] = floor_match.group(1)
        result['total_floors'] = floor_match.group(2)

    # Год постройки
    year_match = re.search(r'Год постройки\s*(\d+)', text)
    if year_match:
        result['build_year'] = year_match.group(1)

    return result

out = []
# Added a check to ensure 'res' is not empty before attempting to iterate over res.id
if not res.empty: # Added check
    for id in set(res.id):
        url = f'https://www.cian.ru/sale/flat/{id}/'
        headers = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
        }

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        adress = ', '.join([val.text.strip() for val in soup.find_all('a', {'data-name': 'AddressItem'})])
        stations = [val.text.strip() for val in soup.find_all('li', {'data-name': 'UndergroundItem'})]
        cleaned_stations = list(set([re.sub(r'\d+\s*мин\.?','', station).strip() for station in stations]))
        text = ', '.join([val.text.strip() for val in soup.find_all('div', {'data-name': 'ObjectFactoidsItem'})]).replace(
            '\xa0', ' ')
        parsed_data = parse_property_info(text)
        out.append([id, adress, cleaned_stations, parsed_data])
    out = pd.DataFrame(out)
    out.columns = ['id', 'adress', 'stations', 'parsed_data']
    res = res.merge(out, on='id', how='inner')
    res.to_csv('cian_raw.csv', index=False)
else: # Added else block for the case where res is empty
    print("The 'res' DataFrame is empty. No property data was scraped from the API.")
    print("Please check the API request parameters or the API endpoint itself.")

In [ ]:
import pandas as pd
import numpy as np
import ast
import re

# Чтение данных
df = pd.read_csv('cian_raw.csv')

# Просмотр структуры данных
print("Исходные колонки:")
print(df.columns.tolist())
print(f"\nРазмер данных: {df.shape}")
print("\nПервые 2 строки:")
print(df.head(2))

# Создаем новый DataFrame с нужной структурой
cleaned_data = []

for idx, row in df.iterrows():
    try:
        # Парсинг parsed_data
        parsed_info = ast.literal_eval(row['parsed_data']) if pd.notna(row['parsed_data']) else {}

        # Извлечение данных
        offer_id = row['id']
        title = f"Квартира {parsed_info.get('total_area', '')}м²"  # Создаем заголовок
        address = row['adress']

        # Извлечение района из адреса
        district_match = re.search(r'р-н (\w+)', address)
        district = district_match.group(1) if district_match else "Не указан"

        # Очистка цены
        price = float(str(row['price']).replace(' ', '').replace('₽', ''))

        # Очистка площади
        total_area = float(str(parsed_info.get('total_area', 0)).replace(',', '.').replace(' ', ''))

        # Предполагаем количество комнат (в реальных данных нужно извлекать из заголовка)
        rooms = 1  # Заглушка - в реальных данных нужно парсить

        # Этаж и всего этажей
        floor = int(parsed_info.get('floor', 0))
        floors_total = int(parsed_info.get('total_floors', 0))

        # Тип дома и год постройки
        house_type = "Не указан"  # В реальных данных нужно парсить
        year_built = int(parsed_info.get('build_year', 0)) if parsed_info.get('build_year') else None

        description = "Описание отсутствует"  # В реальных данных нужно парсить
        link = f"https://www.cian.ru{row['customUrl']}" if pd.notna(row['customUrl']) else ""

        cleaned_data.append({
            'offer_id': offer_id,
            'title': title,
            'address': address,
            'district': district,
            'price': price,
            'total_area': total_area,
            'rooms': rooms,
            'floor': floor,
            'floors_total': floors_total,
            'house_type': house_type,
            'year_built': year_built,
            'description': description,
            'link': link
        })
    except Exception as e:
        print(f"Ошибка обработки строки {idx}: {e}")

# Создаем очищенный DataFrame
df_clean = pd.DataFrame(cleaned_data)

# Удаляем дубликаты
df_clean = df_clean.drop_duplicates(subset=['offer_id'])

# Сохраняем очищенные данные
df_clean.to_csv('cian_clean.csv', index=False, encoding='utf-8')

print(f"Очищенные данные сохранены. Размер: {df_clean.shape}")
print("\nПервые 2 строки очищенных данных:")
print(df_clean.head(2))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Описание набора данных
print("=== ОПИСАНИЕ НАБОРА ДАННЫХ ===")
print(f"Количество строк: {df_clean.shape[0]}")
print(f"Количество столбцов: {df_clean.shape[1]}")
print("\nТипы данных:")
print(df_clean.dtypes)
print("\nПропуски в данных:")
print(df_clean.isnull().sum())
print(f"\nДубликаты: {df_clean.duplicated().sum()}")

# Основные статистики
print("\n=== ОСНОВНЫЕ СТАТИСТИКИ ===")
print(df_clean[['price', 'total_area', 'rooms', 'floor', 'floors_total']].describe())

In [ ]:
# Исследовательский анализ (EDA)

# 1. Распределение цен по районам
plt.figure(figsize=(12, 6))
district_price = df_clean.groupby('district')['price'].mean().sort_values(ascending=False)
district_price.plot(kind='bar')
plt.title('Средняя цена по районам')
plt.xlabel('Район')
plt.ylabel('Средняя цена (руб)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 2. Зависимость цены от площади
plt.figure(figsize=(10, 6))
plt.scatter(df_clean['total_area'], df_clean['price'], alpha=0.6)
plt.title('Зависимость цены от площади')
plt.xlabel('Площадь (м²)')
plt.ylabel('Цена (руб)')
plt.grid(True, alpha=0.3)
plt.show()

# 3. Средняя цена за квадратный метр по районам
df_clean['price_per_m2'] = df_clean['price'] / df_clean['total_area']
district_price_m2 = df_clean.groupby('district')['price_per_m2'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
district_price_m2.plot(kind='bar')
plt.title('Средняя цена за м² по районам')
plt.xlabel('Район')
plt.ylabel('Цена за м² (руб)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 4. Средняя этажность по районам
district_floors = df_clean.groupby('district')['floors_total'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
district_floors.plot(kind='bar')
plt.title('Средняя этажность по районам')
plt.xlabel('Район')
plt.ylabel('Среднее количество этажей')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Дополнительные визуализации

# Гистограмма распределения цен
plt.figure(figsize=(12, 6))
plt.hist(df_clean['price'], bins=30, edgecolor='black', alpha=0.7)
plt.title('Распределение цен на квартиры')
plt.xlabel('Цена (руб)')
plt.ylabel('Количество')
plt.grid(True, alpha=0.3)
plt.show()

# Столбчатая диаграмма средней цены по районам (альтернативный вид)
plt.figure(figsize=(14, 8))
sns.barplot(data=df_clean, x='district', y='price', estimator=np.mean)
plt.title('Средняя цена квартир по районам')
plt.xlabel('Район')
plt.ylabel('Средняя цена (руб)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Создание новых признаков
df_features = df_clean.copy()

# 1. Цена за квадратный метр (уже создали ранее)
df_features['price_per_m2'] = df_features['price'] / df_features['total_area']

# 2. Признак новостройки
df_features['is_new_building'] = (df_features['year_built'] >= 2010).astype(int)

# 3. Возраст здания
current_year = 2024
df_features['age'] = current_year - df_features['year_built']
# Заменяем отрицательные значения на NaN
df_features['age'] = df_features['age'].apply(lambda x: x if x > 0 else None)

# 4. Отношение этажа к общему количеству этажей
df_features['floor_ratio'] = df_features['floor'] / df_features['floors_total']

# 5. Расстояние до центра (условно, на основе района)
center_districts = ['Тверской', 'Арбат', 'Хамовники', 'Якиманка']
df_features['center_distance'] = df_features['district'].apply(
    lambda x: 0 if x in center_districts else 1 if x != 'Не указан' else 2
)


# 6. Категория площади
def area_category(area):
    if area < 30:
        return 'small'
    elif area < 60:
        return 'medium'
    elif area < 100:
        return 'large'
    else:
        return 'very_large'


df_features['area_category'] = df_features['total_area'].apply(area_category)


# 7. Этаж категория
def floor_category(floor, total_floors):
    if floor == 1:
        return 'first'
    elif floor == total_floors:
        return 'last'
    else:
        return 'middle'


df_features['floor_category'] = df_features.apply(
    lambda x: floor_category(x['floor'], x['floors_total']), axis=1
)

print("=== НОВЫЕ ПРИЗНАКИ ===")
print(df_features[['price_per_m2', 'is_new_building', 'age', 'floor_ratio',
                   'center_distance', 'area_category', 'floor_category']].head())

In [ ]:
# Агрегированные показатели
print("=== АГРЕГИРОВАННЫЕ ПОКАЗАТЕЛИ ===")

# Средняя цена по районам
avg_price_by_district = df_features.groupby('district').agg({
    'price': ['mean', 'median', 'count'],
    'price_per_m2': 'mean',
    'total_area': 'mean',
    'age': 'mean'
}).round(2)

avg_price_by_district.columns = ['Средняя цена', 'Медианная цена', 'Количество',
                                 'Средняя цена м²', 'Средняя площадь', 'Средний возраст']
print("Средние показатели по районам:")
print(avg_price_by_district)

# Статистика по типам площади
print("\nСтатистика по категориям площади:")
area_stats = df_features.groupby('area_category').agg({
    'price': 'mean',
    'price_per_m2': 'mean',
    'total_area': 'mean'
}).round(2)
print(area_stats)

# Корреляционный анализ
print("\nКорреляционная матрица:")
numeric_cols = ['price', 'total_area', 'rooms', 'floor', 'floors_total',
                'price_per_m2', 'age', 'floor_ratio']
correlation_matrix = df_features[numeric_cols].corr()
print(correlation_matrix.round(3))

# Финальные визуализации и выводы

# Топ-5 самых дорогих районов
top_expensive = avg_price_by_district.nlargest(5, 'Средняя цена м²')
print("=== ТОП-5 САМЫХ ДОРОГИХ РАЙОНОВ ===")
print(top_expensive[['Средняя цена м²', 'Средняя цена']])

# Визуализация корреляций
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=0.5)
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

# Сравнение новостроек и вторичного жилья
if 'is_new_building' in df_features.columns:
    new_vs_old = df_features.groupby('is_new_building').agg({
        'price': 'mean',
        'price_per_m2': 'mean',
        'total_area': 'mean'
    }).round(2)
    new_vs_old.index = ['Вторичка', 'Новостройка']
    print("\nСравнение новостроек и вторичного жилья:")
    print(new_vs_old)